In [1]:
# Cell 1 (no SciPy): imports and basic settings
import numpy as np
from math import pi

# For simple linear algebra we use numpy.linalg
import numpy.linalg as nla

# Optional packages: emcee for MCMC, astropy for accurate distances
try:
    import emcee
    has_emcee = True
except Exception:
    has_emcee = False

try:
    from astropy.cosmology import FlatLambdaCDM
    has_astropy = True
    # example fiducial cosmology
    cosmo = FlatLambdaCDM(H0=70.0, Om0=0.3)
except Exception:
    has_astropy = False
    cosmo = None

print("numpy OK, numpy.linalg OK, emcee available:", has_emcee, "astropy available:", has_astropy)




numpy OK, numpy.linalg OK, emcee available: False astropy available: False


In [11]:
# Cell 2: theory mapping: delta0 <-> epsilon_bar
# Transfer coefficients (guides)
beta0 = 0.5     # for delta_rho = beta0 * eps^2
k_beta = 0.3    # for birefringence beta = k_beta * eps^2 (radians)
c_f = 0.5       # for fractional growth shift

def delta_from_epsilon(eps_bar, beta0=beta0):
    return beta0 * eps_bar**2

def epsilon_from_delta(delta0, beta0=beta0):
    return np.sqrt(delta0 / beta0)

def birefringence_from_epsilon(eps_bar, k_beta=k_beta):
    return k_beta * eps_bar**2  # radians


In [12]:
# Cell 3: generate simple standard siren mock
def generate_gw_mocks(N=50, zmin=0.01, zmax=0.5, sigma_frac=0.02, seed=0):
    rng = np.random.default_rng(seed)
    zs = rng.uniform(zmin, zmax, size=N)
    # fiducial cosmology: assume H0=70, Omega_m=0.3 (simple FLRW distances)
    H0 = 70.0
    c = 299792.458  # km/s
    # simple flat LCDM approximate D_L (small z) for demo: D_L ~ c/H0 * z(1 + 0.5(1-q0)z)
    q0 = -0.55
    DL_fid = (c/H0) * zs * (1 + 0.5*(1-q0)*zs)
    # add observational noise
    sigs = sigma_frac * DL_fid
    DL_obs = DL_fid * (1 + rng.normal(scale=sigma_frac, size=N))
    return zs, DL_obs, sigs

zs, DL_obs, DL_sig = generate_gw_mocks()
print("Generated GW mocks:", len(zs))


Generated GW mocks: 50


In [13]:
# Cell 4: simple chi2 likelihood and Fisher estimate for single param delta0
def model_DL_EM(z):
    # use same fiducial DL as in mock generator
    H0 = 70.0; c=299792.458; q0=-0.55
    return (c/H0) * z * (1 + 0.5*(1-q0)*z)

def chi2_delta0(delta0, zs, DL_obs, DL_sig):
    # Here model: DL_GW = DL_EM * (1 + delta_rho(z)), with delta_rho=delta0 (constant model)
    model = model_DL_EM(zs) * (1 + delta0)
    invvar = 1.0 / (DL_sig**2)
    return np.sum((DL_obs - model)**2 * invvar)

# Fisher approximation: F = 0.5 * d2 chi2 / d delta0^2 ~ sum ( (d model/d delta0)^2 / sigma^2 )
dmodel_ddelta = model_DL_EM(zs)  # derivative wrt delta0
F = np.sum((dmodel_ddelta**2) / (DL_sig**2))
sigma_delta0 = 1.0 / np.sqrt(F)
print("Fisher sigma(delta0) (GW only):", sigma_delta0)
# translate to epsilon_bar assuming delta0 = beta0 * eps^2 (propagate)
sigma_eps = 0.5 * sigma_delta0 / np.sqrt(beta0 * max(1e-12, 1e-12 + 1e-12))  # placeholder; see next cell for correct propagation


Fisher sigma(delta0) (GW only): 0.00282842712474619


In [14]:
# Cell 5: propagate sigma(delta0) -> sigma(eps_bar) via eps = sqrt(delta0/beta0)
def propagate_sigma_eps(delta0, sigma_delta0, beta0=beta0):
    # use delta0 >> sigma approx: sigma_eps = sigma_delta0 / (2 * beta0 * eps)
    eps = np.sqrt(delta0 / beta0)
    if eps == 0:
        return np.inf
    sigma_eps = sigma_delta0 / (2.0 * beta0 * eps)
    return eps, sigma_eps

# Example: set delta0 = 1e-4 (hypothetical)
delta0_example = 1e-4
# We need sigma_delta0 from Fisher; use value found above (note: for our toy mocks sigma ~ large)
sigma_delta0_fid = sigma_delta0
eps_est, sigma_eps_est = propagate_sigma_eps(delta0_example, sigma_delta0_fid)
print("Example: delta0=",delta0_example," -> eps=",eps_est," sigma_eps=",sigma_eps_est)


Example: delta0= 0.0001  -> eps= 0.01414213562373095  sigma_eps= 0.19999999999999998


In [15]:
# Cell 6 (fixed): quick CMB birefringence check and comparison to experimental limits
import numpy as np

# Example transfer coefficient and eps value (use your chosen values or compute them)
k_beta = 0.3         # transfer coefficient (radians per eps^2)
eps_bar = 1e-2       # example eps_bar

def birefringence_from_epsilon(eps_bar, k_beta=k_beta):
    """Return birefringence angle in radians."""
    return k_beta * eps_bar**2

def rad2deg(x):
    return x * 180.0 / np.pi

beta_rad = birefringence_from_epsilon(eps_bar, k_beta=k_beta)
beta_deg = rad2deg(beta_rad)

# Experimental reference limits (approximate)
aliCPT_limit_deg = 0.04   # AliCPT single-season example limit (degrees)
simons_limit_deg = 0.01   # optimistic future limit (degrees)

print(f"eps_bar = {eps_bar:e}, k_beta = {k_beta:.3g}")
print(f"Predicted birefringence: {beta_rad:.3e} rad = {beta_deg:.4e} deg")
print(f"AliCPT example limit: {aliCPT_limit_deg:.3e} deg")
print(f"Simons/LiteBIRD optimistic limit: {simons_limit_deg:.3e} deg")

# Quick interpretative check
if beta_deg < simons_limit_deg:
    print("Predicted signal below optimistic future limits (not detectable with current/forthcoming CMB polarimeters).")
elif beta_deg < aliCPT_limit_deg:
    print("Predicted signal below AliCPT current constraint but near future sensitivity.")
else:
    print("Predicted signal above current AliCPT limits (would be detectable).")


eps_bar = 1.000000e-02, k_beta = 0.3
Predicted birefringence: 3.000e-05 rad = 1.7189e-03 deg
AliCPT example limit: 4.000e-02 deg
Simons/LiteBIRD optimistic limit: 1.000e-02 deg
Predicted signal below optimistic future limits (not detectable with current/forthcoming CMB polarimeters).


In [7]:
# Cell 7: instructions
print("""
To run a full forecast:
1) Replace the toy DL model by a full numerical FLRW distance (astropy.cosmology or CAMB outputs).
2) Load realistic covariance matrices (Euclid/LSST, CMB instrumental noise).
3) Build full joint likelihood and run Fisher or MCMC (emcee or cobaya).
4) Use Appendix H linearisation code to compute beta0,k_beta,c_f for your rho profile, then map delta0 <-> epsilon_bar for reporting.
The repository contains example covariance files and scripts to reproduce Table 1 in the manuscript.
""")



To run a full forecast:
1) Replace the toy DL model by a full numerical FLRW distance (astropy.cosmology or CAMB outputs).
2) Load realistic covariance matrices (Euclid/LSST, CMB instrumental noise).
3) Build full joint likelihood and run Fisher or MCMC (emcee or cobaya).
4) Use Appendix H linearisation code to compute beta0,k_beta,c_f for your rho profile, then map delta0 <-> epsilon_bar for reporting.
The repository contains example covariance files and scripts to reproduce Table 1 in the manuscript.

